In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Import/ install relevant libraries

In [ ]:
import zipfile
import numpy as np
import pandas as pd
import string
import spacy
import re
from nltk.tokenize import word_tokenize
from textblob import TextBlob
import emoji
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import lightgbm as lgb 
from sklearn.metrics import accuracy_score, f1_score
from imblearn.over_sampling import SMOTE

In [ ]:
with zipfile.ZipFile("/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip", 'r') as zip_ref:
    zip_ref.extractall("./")
with zipfile.ZipFile("/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip", 'r') as zip_ref:
    zip_ref.extractall("./")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
train_df = pd.read_csv("./train.csv")
test_df = pd.read_csv("./test.csv")
train_df.head()

# 2. Data Cleaning

## 2.1 Removing irrelevant columns

In [ ]:
train_df.drop(["id"], axis=1, inplace=True)

In [ ]:
test_df.drop(["id"], axis=1, inplace=True)

## 2.2 Removing new line character

In [ ]:
def remove_newline(text):
    return text.replace("\n", " ")

In [ ]:
train_df["comment_text"] = train_df["comment_text"].apply(remove_newline)
train_df.head()

In [ ]:
test_df["comment_text"] = test_df["comment_text"].apply(remove_newline)
test_df.head()

## 2.3 Removing Emojis

In [ ]:
def removeEmojis(text):
    allchars = [c for c in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI["en"]]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

In [ ]:
train_df["comment_text"] = train_df["comment_text"].apply(removeEmojis)
train_df.head()

In [ ]:
test_df["comment_text"] = test_df["comment_text"].apply(removeEmojis)
test_df.head()

## 2.4 Removing website URLs

In [ ]:
def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

In [ ]:
train_df["comment_text"] = train_df["comment_text"].apply(strip_links)
train_df.head()

In [ ]:
test_df["comment_text"] = test_df["comment_text"].apply(strip_links)
test_df.head()

## 2.5 Uncaptializing 

In [ ]:
def uncapitalize(text):
    return text.lower()

In [ ]:
train_df["comment_text"] = train_df["comment_text"].apply(uncapitalize)
train_df.head()

In [ ]:
test_df["comment_text"] = test_df["comment_text"].apply(uncapitalize)
test_df.head()

## 2.6 Expanding abberviations

In [ ]:
contraction_map = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"}

In [ ]:
def expand_abbr(text):
    new_text = text
    for item in contraction_map:
        if item in text:
            new_text = new_text.replace(item,contraction_map[item])
    return new_text

In [ ]:
train_df["comment_text"] = train_df["comment_text"].apply(expand_abbr)
train_df.head()

In [ ]:
test_df["comment_text"] = test_df["comment_text"].apply(expand_abbr)
test_df.head()

## 2.7 Removing stopwords

In [ ]:
sp = spacy.load('en_core_web_sm')
all_stopwords = sp.Defaults.stop_words

In [ ]:
def remove_stopwords(text):
    #function returns list of words per text_comment that are not stopwords
    text_tokens = word_tokenize(text)
    tokens_without_sw= [word for word in text_tokens if not word in all_stopwords]
    return tokens_without_sw

In [ ]:
train_df["comment_text"] = train_df["comment_text"].apply(remove_stopwords)
train_df.head()

In [ ]:
test_df["comment_text"] = test_df["comment_text"].apply(remove_stopwords)
test_df.head()

## 2.8 Removing punctuation

In [ ]:
all_punctuation = set(string.punctuation)
all_punctuation.add("...")
all_punctuation.add('’')
all_punctuation.add('-')
all_punctuation.add('“')
all_punctuation.add('[')
all_punctuation.add(']')
all_punctuation.add('==')

In [ ]:
all_punctuation

In [ ]:
def remove_punctuation(token_list):
    new_list = []
    for tok in token_list:
        if tok not in all_punctuation:
            new_list.append(tok)
    return new_list

In [ ]:
train_df["comment_text"] = train_df["comment_text"].apply(remove_punctuation)
train_df.head()

In [ ]:
test_df["comment_text"] = test_df["comment_text"].apply(remove_punctuation)
test_df.head()

# 3. Getting word embeddings with GloVe

In [ ]:
word_embeddings = {}
f = open('/kaggle/input/glove6b/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs

In [ ]:
def get_sentence_vectors(text):
    sentence_vector = np.zeros((100,))
    if len(text)==0:
        return sentence_vector
    for word_token in text:
        if word_token in word_embeddings:
            sentence_vector += word_embeddings[word_token]
    sentence_vector = sentence_vector/len(text)
    return sentence_vector

In [ ]:
train_df["comment_text"] = train_df["comment_text"].apply(get_sentence_vectors)
train_df.head()

In [ ]:
test_df["comment_text"] = test_df["comment_text"].apply(get_sentence_vectors)
test_df.head()

# 4. Preparing train and test set

In [ ]:
train_data = train_df["comment_text"]
test_data = test_df["comment_text"]
label=train_df[['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']]

In [ ]:
x_train, x_test, y_train, y_test =  train_test_split(train_data, label, test_size = 0.3, random_state=123)

In [ ]:
label.columns.to_list()

# 5. LightGBM

In [ ]:
accuracy_dict = {}
for category in label.columns.to_list():
    lgb_clf = lgb.LGBMClassifier()
    lgb_clf.fit(x_train.to_list(), y_train[category])
    y_pred = lgb_clf.predict(x_test.to_list())
    accuracy_dict[category] = accuracy_score(y_test[category], y_pred)*100

# 6. Accuracy Evaluation

In [ ]:
for category,score in accuracy_dict.items():
    print(f"{category}: {score}")